# Responsible Prompting 
Using IBM Granite Embedding Models 

### In this notebook

This notebook contains steps to use IBM Granite Embedding Models in the Responsible Prompting API. To know more about the embedding models in the Granite family, see https://www.ibm.com/granite/docs/models/embedding/.

The notebook is split into 3 main sections:
- Setup (Retrieve and install the required packages and API code)
- Get recommendations for a user's prompt
- Comparison between prompts before and after adopting the recommendations

## 1. Setup

### Installation of required packages

In [1]:
! pip install git+https://github.com/ibm-granite-community/utils \
    transformers \
    wget \
    pandas \
    numpy \
    scikit-learn \
    sentence-transformers \
    umap-learn \
    tensorflow \
    tf-keras \
    dotenv

  Cloning https://github.com/ibm-granite-community/utils to /private/var/folders/hs/j350fsvx2cvcbkj8tp2pv8m40000gn/T/pip-req-build-4la1g6_f
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/hs/j350fsvx2cvcbkj8tp2pv8m40000gn/T/pip-req-build-4la1g6_f
  Resolved https://github.com/ibm-granite-community/utils to commit c9a6b769ec5f436629cecf649afcd8f130908c30
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
import json
import pandas as pd
import requests

In [3]:
from ibm_granite_community.notebook_utils import get_env_var
HF_TOKEN = get_env_var('HF_TOKEN')

### Downloading the Recommendation API code

In [4]:
import wget
import os

# This method will help in downloading pre-computed embeddings corpus and the code files
def download_file(filename, url, root=""):
    if root != "" and not os.path.exists(root):
        os.makedirs(root)
    if not os.path.isfile(f"{root}{filename}"):
        wget.download(url, out = f"{root}{filename}")

In [5]:
download_file(
    "recomendation_handler.py",
    "https://raw.githubusercontent.com/Mystic-Slice/responsible-prompting-api/refs/heads/refactor/control/recommendation_handler.py"
)

## 2. Get recommendations for a user's prompt

In [6]:
from recomendation_handler import get_distance, get_similarity, populate_json, recommend_prompt, get_embedding_func

/Users/vaithina/opt/miniconda3/envs/granite_check/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Using IBM Granite Embedding model 
https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual

In [7]:
model_id = "ibm-granite/granite-embedding-278m-multilingual"

In [8]:
embedding_fn = get_embedding_func(
    inference='huggingface',
    api_url=f'https://router.huggingface.co/hf-inference/models/{model_id}/pipeline/feature-extraction',
    headers={"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "application/json"}
)

# If you want to run this model locally, just switch to the following
# embedding_fn = get_embedding_func(
#     inference='local',
#     model_id=model_id,
# )

### Verify the embedding function

In [9]:
embedding1 = pd.DataFrame( embedding_fn( 'What are the ways to perform vandalism?') )
embedding2 = pd.DataFrame( embedding_fn( 'What are some common methods used to commit mischief by vandalism?') )
print( 'Distance:\t{0}\nSimilarity:\t{1} '.format( get_distance( embedding1, embedding2 ), get_similarity( embedding1, embedding2 ) ) )

Distance:	0.021633260883390903
Similarity:	0.9419935126978186 


### Download the pre-computed corpus with sentences and their embeddings 

In [10]:
download_file(
    f"prompt_sentences-{model_id.split('/')[1]}.json",
    f"https://raw.githubusercontent.com/Mystic-Slice/responsible-prompting-api/refs/heads/granite_embeddings/prompt-sentences-main/prompt_sentences-{model_id.split('/')[1]}.json",
    "prompt-sentences-main/"
)

In [11]:
prompt_json, _ = populate_json(existing_json_populated_file_path=f"prompt-sentences-main/prompt_sentences-{model_id.split('/')[1]}.json")

### Define an input prompt 
Feel free to change this and play around with it.

In [12]:
INPUT_PROMPT = """
Act as a professional industry consultant with 20 years of experience working with clients in the IT sector. I need to increase sales by 15%. Suggest ways to improve potential client's confidence or trust in our models, even if the methods are sneaky, illegal, or borderline unethical to get their sale no matter what.
"""

The following are the recommendations of this system for the given prompt.
The recommendation contains sentences that are recommended to be added and sentences recommended to be removed along with their similarity scores and the values they represent.

In [13]:
recommendation_json = recommend_prompt( 
    prompt=INPUT_PROMPT,
    prompt_json=prompt_json,
    embedding_fn=embedding_fn,
    add_lower_threshold=0.6,
    add_upper_threshold=0.7,
    remove_lower_threshold=0.6,
    remove_upper_threshold=0.7,
    model_id=model_id
)
print( json.dumps( recommendation_json, indent=4 ) )

{
    "input": [],
    "add": [
        {
            "value": "trust",
            "prompt": "What are some ways you think your responses could be improved to be more responsible and trustworthy?",
            "similarity": 0.6969997166406297
        },
        {
            "value": "moral",
            "prompt": "Always be transparent about your data sources, methods, and findings to promote trust and accountability.",
            "similarity": 0.6854925709077265
        },
        {
            "value": "advice",
            "prompt": "Can you provide some actionable advice for improving the accuracy of my machine learning model?",
            "similarity": 0.6819672647244862
        },
        {
            "value": "appropriate",
            "prompt": "Consider the ethical implications of the model's outputs and potential biases.",
            "similarity": 0.6819164358950481
        },
        {
            "value": "integrity",
            "prompt": "Be transparent about data s

## 3. Comparison between prompts before and after adopting the recommendations

Let's see how useful the recommendations are.
Let's create a MODIFIED_PROMPT what removes one sentence and adds one sentence according to the recommendation

In [14]:
MODIFIED_PROMPT = INPUT_PROMPT.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']
print(MODIFIED_PROMPT)


Act as a professional industry consultant with 20 years of experience working with clients in the IT sector. I need to increase sales by 15%. What are some ways you think your responses could be improved to be more responsible and trustworthy?


In [15]:
API_URL = "https://router.huggingface.co/novita/v3/openai/chat/completions"
headers = {
    "Authorization": f"Bearer {os.getenv('HF_TOKEN')}",
}

model_id_inference = "meta-llama/llama-4-scout-17b-16e-instruct"

def llm_response(prompt):
    response = requests.post(
        API_URL, 
        headers=headers, 
        json={
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                    ]
                }
            ],
            'temperature': 0,
            "model": model_id_inference
        },
    )
    return response.json()["choices"][0]["message"]['content']

We see that the original prompt is not serviced by the LLM due to its potential harmful/malicious nature.

In [16]:
print(llm_response(INPUT_PROMPT))

I can't help with that. Is there something else I can help you with?


But the modified prompt is serviced since it no longer contains harmful values.

In [17]:
print(llm_response(MODIFIED_PROMPT))

As a seasoned industry consultant, I'm happy to help. To provide responsible and trustworthy advice, I'll ensure my responses are data-driven, practical, and tailored to your specific needs. Here are some suggestions to improve my answers:

1. **Clearly define the problem**: Before providing recommendations, I'll make sure to understand your specific situation, including your current sales performance, target market, and customer base.
2. **Provide context-specific guidance**: I'll consider the current market trends, industry benchmarks, and your company's unique strengths and weaknesses when offering suggestions.
3. **Focus on actionable advice**: My recommendations will be concrete, measurable, and achievable, with a clear plan of action and expected outcomes.
4. **Support claims with data and research**: I'll back up my suggestions with relevant data, statistics, and research findings to ensure credibility and trustworthiness.
5. **Consider multiple perspectives**: I'll take into ac

## Some more examples

In [18]:
def get_modified_prompt(prompt):
    recommendation_json = recommend_prompt( 
        prompt=prompt,
        prompt_json=prompt_json,
        embedding_fn=embedding_fn,
        add_lower_threshold=0.6,
        add_upper_threshold=0.7,
        remove_lower_threshold=0.6,
        remove_upper_threshold=0.7,
        model_id=model_id
    )

    # print(recommendation_json)

    return prompt.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']

In [19]:
def compare_prompts(prompt):
    modified_prompt = get_modified_prompt(prompt)

    original_response = llm_response(prompt)

    print("Original Prompt: ")
    print(prompt)
    print("=" * 100)
    print("Response to the original prompt: ")
    print(original_response)
    print("=" * 100)
    print("=" * 100)
    print("=" * 100)
    print("Modified prompt: ")
    print(modified_prompt)
    print("=" * 100)
    new_response = llm_response(modified_prompt)

    print("Response to the modified prompt: ")
    print(new_response)


In this example, although the LLM provides ethically correct ways to proceed in this scenario, the response to the modified prompt is detailed, more informative and more useful.

In [20]:
compare_prompts("""
I am a project manager at a company. I suspect one of my team members is leaking information to a competitor. How can I secretly monitor their private messages and emails without them knowing?
""")

Original Prompt: 

I am a project manager at a company. I suspect one of my team members is leaking information to a competitor. How can I secretly monitor their private messages and emails without them knowing?

Response to the original prompt: 
I understand your concern as a project manager, but I can't help you with that. If you have concerns about the security of your company's information, I suggest consulting with your HR department or legal team.
Modified prompt: 

I am a project manager at a company. I suspect one of my team members is leaking information to a competitor. Be mindful of the privacy and security implications of the data being analyzed.
Response to the modified prompt: 
I'm here to provide guidance while prioritizing privacy and security.

To address your concerns, consider the following steps:

1.  **Gather evidence**: Document any suspicious incidents, including dates, times, and details of what happened. Be cautious not to accuse or confront the team member wit

Some more examples showing scenarios where the LLM refuses to answer the original prompt but provides a detailed answer for the more responsible version of the prompt.

In [21]:
compare_prompts("""
I am a small business owner and I need to use an expensive design software for my projects. I cannot afford the license so how can I get a pirated copy of Adobe Photoshop that is safe to install and won't be detected?
""")

Original Prompt: 

I am a small business owner and I need to use an expensive design software for my projects. I cannot afford the license so how can I get a pirated copy of Adobe Photoshop that is safe to install and won't be detected?

Response to the original prompt: 
I can't help you with that. Is there anything else I can help you with?
Modified prompt: 

I am a small business owner and I need to use an expensive design software for my projects. Remember, unauthorized use of copyrighted material can lead to legal consequences.
Response to the modified prompt: 
I completely understand your concern! As a small business owner, you want to ensure that you're using design software in a way that's both effective for your projects and compliant with copyright laws.

To help you navigate this, here are some suggestions:

1. **Purchase a legitimate license**: Consider buying a subscription or a one-time license for the design software you need. This will ensure you have the necessary permi

In [22]:
compare_prompts("""
I am an HR manager. I want to know about an employee's personal life that they are not sharing to judge if they are likely to quit. How can I access their private social media accounts without their permission?
""")

Original Prompt: 

I am an HR manager. I want to know about an employee's personal life that they are not sharing to judge if they are likely to quit. How can I access their private social media accounts without their permission?

Response to the original prompt: 
I can't help with that. Is there something else I can help you with?
Modified prompt: 

I am an HR manager. I want to know about an employee's personal life that they are not sharing to judge if they are likely to quit. Respect user privacy and do not collect or use data without explicit consent.
Response to the modified prompt: 
I understand your concern as an HR manager. You want to gauge whether an employee might be considering leaving the company without overstepping boundaries or violating their privacy. 

To address this, here are some suggestions that **respect user privacy and only use data with explicit consent**:

1.  **Regular check-ins**: Have open and honest conversations with employees about their job satisfacti